In [84]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

**loading the dataset**

In [54]:
from google.colab import files
uploaded = files.upload()

Saving customer_churn.csv.csv to customer_churn.csv (5).csv


In [55]:
data = pd.read_csv('customer_churn.csv.csv')

**Dataset exploration**

In [56]:
print(data.head())
print(data.info())

   CustomerID  Tenure  MonthlyCharges  TotalCharges        Contract  \
0        1001       5            70.0         350.0  Month-to-month   
1        1002      10            85.5         850.5        Two year   
2        1003       3            55.3         165.9        One year   
3        1004       8            90.0         720.0  Month-to-month   
4        1005       2            65.2         130.4        One year   

      PaymentMethod  Churn  
0  Electronic check      1  
1      Mailed check      0  
2  Electronic check      1  
3       Credit card      0  
4  Electronic check      1  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CustomerID      5 non-null      int64  
 1   Tenure          5 non-null      int64  
 2   MonthlyCharges  5 non-null      float64
 3   TotalCharges    5 non-null      float64
 4   Contract        5 non-nu

**Handling missing values**

In [57]:
data = data.dropna()   #dropping rows with missing values

**Encoding categorical variables**

In [58]:
data = pd.get_dummies(data, drop_first=True)
print(data.head())

   CustomerID  Tenure  MonthlyCharges  TotalCharges  Churn  Contract_One year  \
0        1001       5            70.0         350.0      1              False   
1        1002      10            85.5         850.5      0              False   
2        1003       3            55.3         165.9      1               True   
3        1004       8            90.0         720.0      0              False   
4        1005       2            65.2         130.4      1               True   

   Contract_Two year  PaymentMethod_Electronic check  \
0              False                            True   
1               True                           False   
2              False                            True   
3              False                           False   
4              False                            True   

   PaymentMethod_Mailed check  
0                       False  
1                        True  
2                       False  
3                       False  
4               

**data splitting**

In [59]:
from sklearn.model_selection import train_test_split

X = data.drop('Churn', axis=1)
y = data['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Developing and training the model**

In [75]:
class ChurnModel(nn.Module):
    def __init__(self):
        super(ChurnModel, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1],64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = nn.functional.dropout(x, 0.5, training=self.training)
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

model = ChurnModel()

**model training**

In [79]:
X_train = X_train.astype(float)
X_test = X_test.astype(float)

In [76]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop (simplified example)
for epoch in range(10):
    model.train()
    optimizer.zero_grad()
    outputs = model(torch.tensor(X_train.values).float())
    loss = criterion(outputs.squeeze(), torch.tensor(y_train.values).float())
    loss.backward()
    optimizer.step()

**model evaluation**

In [80]:
model.eval()
outputs = model(torch.tensor(X_test.values).float())
predictions = (outputs.squeeze().detach().numpy() > 0.5).astype(int)
accuracy = np.mean(predictions == y_test.values)
print(f'Test accuracy: {accuracy}')

Test accuracy: 0.0


**model optimization**

In [86]:

quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

/tmp/ipython-input-2887292789.py:1: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  quantized_model = torch.quantization.quantize_dynamic(


**preparing model for deployment**

In [87]:
torch.save(model.state_dict(), 'churn_model.pth')